# Portfolio Optimization

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
import cvxpy as cp

In [4]:
R = []
assets = ["MSFT","AAPL","BEP"]
for stock in assets:
    data = yf.Ticker(stock)
    df = data.history(period="10y")
    close = df['Close'].values
    returns = (close[1:] - close[:-1])/close[:-1]
    R.append(returns)

R = np.array(R).T
r = np.mean(R,axis=0)
S = np.cov(R.T)

In [5]:
n = len(assets)
q = 0.1
e = np.ones(n)
X = cp.Variable(n)
obj = cp.Minimize(cp.quad_form(X,S) - q*r@X)
constraints = [e@X == 1, X >= 0]
problem = cp.Problem(obj,constraints)
result = problem.solve()
X.value

array([0.44462007, 0.25554628, 0.29983365])

## Model with Transaction Costs

\begin{align*}
    \min_{\mathbf{x}, \mathbf{u}_{\text{buy}}, \mathbf{u}_{\text{sell}}} & \quad q\cdot \mathbf{x}^T \Sigma \mathbf{x} + \mathbf{r}^T \mathbf{x}\\
    \text{subject to:} & \\
    & \mathbf{x} = \mathbf{x}_{\text{init}} + \mathbf{u}_{\text{buy}} - \mathbf{u}_{\text{sell}} \\
    & \mathbf{u}_{\text{buy}} \ge 0, \quad \mathbf{u}_{\text{sell}} \ge 0 \\
    & (1 - f_{\text{sell}}) \mathbf{1}^T \mathbf{u}_{\text{sell}} = (1 + f_{\text{buy}}) \mathbf{1}^T \mathbf{u}_{\text{buy}} \\
    & \mathbf{1}^T \mathbf{x} = 1
\end{align*}

In [13]:
def optimize_with_transaction_costs(x_init, f_buy, f_sell, q, S, r, n):
    """
    Optimize portfolio weights with transaction costs.
    Args:
        x_init (np.array): Initial portfolio weights.
        f_buy (float): Buying transaction cost.
        f_sell (float): Selling transaction cost.
        q (float): Risk aversion parameter.
        S (np.array): Covariance matrix of returns.
        r (np.array): Expected returns.
        n (int): Number of assets.
    Returns:
        tuple: Optimized portfolio weights, buy adjustments, sell adjustments.
    """
    e = np.ones(n)
    u_buy = cp.Variable(n)
    u_sell = cp.Variable(n)
    x = x_init + u_buy - u_sell

    objective = cp.Minimize(q * cp.quad_form(x, S) - r @ x)
    constraints = [
        x == x_init + u_buy - u_sell,
        x >= 0,
        u_buy >= 0,
        u_sell >= 0,
        (1 - f_sell) * e @ u_sell == (1 + f_buy) * e @ u_buy,
        e @ x == 1
    ]

    problem = cp.Problem(objective, constraints)
    result = problem.solve()

    return x.value, u_buy.value, u_sell.value

In [ ]:
x_init = np.array([0.4, 0.3, 0.3])
f_buy = 0.00000
f_sell = 0.00001
q = 1
optimized_x, optimized_u_buy, optimized_u_sell = optimize_with_transaction_costs(x_init, f_buy, f_sell, q, S, r, n)
print(optimized_x)

[ 8.67190298e-01  1.32811091e-01 -6.13873002e-06]
